In [1]:
from deeplab import common, model
from deeplab.utils import train_utils
from deeplab.core import utils

from tensorflow.keras.utils import OrderedEnqueuer
from tensorflow.contrib import slim as contrib_slim
from slim.nets import resnet_utils

import tensorflow as tf
print(tf.__version__)

from lib import dataloader, modelbuilder, lr_schedular

from albumentations import *
import cv2
import numpy as np

# %config IPCompleter.greedy=True

%load_ext autoreload
%autoreload 2

flags = tf.app.flags
FLAGS = flags.FLAGS
flags.DEFINE_string('f', '', 'kernel')

1.12.3


## set parameters

In [2]:
# model parameters
flags.DEFINE_boolean('fine_tune_batchnorm', True, 'fine tune batchnorm')
flags.DEFINE_float('weight_decay', 1e-4, 'weight decay')

flags.DEFINE_enum('optimizer', 'momentum', ['momentum', 'adam'],
                  'Which optimizer to use.')

# Momentum optimizer flags

flags.DEFINE_enum('learning_policy', 'poly', ['poly', 'step'],
                  'Learning rate policy for training.')

# Use 0.007 when training on PASCAL augmented training set, train_aug. When
# fine-tuning on PASCAL trainval set, use learning rate=0.0001.
flags.DEFINE_float('base_learning_rate', .0001,
                   'The base learning rate for model training.')

flags.DEFINE_float('decay_steps', 0.0,
                   'Decay steps for polynomial learning rate schedule.')

flags.DEFINE_float('end_learning_rate', 0.0,
                   'End learning rate for polynomial learning rate schedule.')

flags.DEFINE_float('learning_rate_decay_factor', 0.1,
                   'The rate to decay the base learning rate.')

flags.DEFINE_integer('learning_rate_decay_step', 2000,
                     'Decay the base learning rate at a fixed step.')

flags.DEFINE_float('learning_power', 0.9,
                   'The power value used in the poly learning policy.')

flags.DEFINE_integer('training_number_of_steps', 3000,
                     'The number of steps used for training')

flags.DEFINE_float('momentum', 0.9, 'The momentum value to use')

# Adam optimizer flags
flags.DEFINE_float('adam_learning_rate', 0.001,
                   'Learning rate for the adam optimizer.')
flags.DEFINE_float('adam_epsilon', 1e-08, 'Adam optimizer epsilon.')

# slow start
flags.DEFINE_integer('slow_start_step', 0,
                     'Training model with small learning rate for few steps.')

flags.DEFINE_float('slow_start_learning_rate', 1e-4,
                   'Learning rate employed during slow start.')

# one cycle policy parameter


In [3]:
crop_size = [256, 256]
outputs_to_num_classes = {'semantic': 3}

model_options = common.ModelOptions(
    outputs_to_num_classes,
    crop_size,
    output_stride=16
)._replace(
    add_image_level_feature=True,
    aspp_with_batch_norm=True,
    aspp_with_separable_conv=False,
    decoder_use_separable_conv=False,
    decoder_output_is_logits=True,
    logits_kernel_size=1,
    decoder_output_stride=[2],
    multi_grid=[1,2],
    atrous_rates=[2,4,8],
    model_variant='resnet_mod') 

for key, value in model_options._asdict().items():
    print(key, value)

outputs_to_num_classes {'semantic': 3}
crop_size [256, 256]
atrous_rates [2, 4, 8]
output_stride 16
preprocessed_images_dtype <dtype: 'float32'>
merge_method max
add_image_level_feature True
image_pooling_crop_size None
image_pooling_stride [1, 1]
aspp_with_batch_norm True
aspp_with_separable_conv False
multi_grid [1, 2]
decoder_output_stride [2]
decoder_use_separable_conv False
logits_kernel_size 1
model_variant resnet_mod
depth_multiplier 1.0
divisible_by None
prediction_with_upsampled_logits True
dense_prediction_cell_config None
nas_architecture_options {'nas_stem_output_num_conv_filters': 20, 'nas_use_classification_head': False, 'nas_remove_os32_stride': False}
use_bounded_activation False
aspp_with_concat_projection True
aspp_with_squeeze_and_excitation False
aspp_convs_filters 256
decoder_use_sum_merge False
decoder_filters 256
decoder_output_is_logits True
image_se_uses_qsigmoid False
label_weights 1.0
sync_batch_norm_method None
batch_norm_decay 0.9997


In [4]:
for key in FLAGS.__flags.keys():
    print('  {}: {}'.format(key, getattr(FLAGS, key)))

  min_resize_value: None
  max_resize_value: None
  resize_factor: None
  keep_aspect_ratio: True
  logits_kernel_size: 1
  model_variant: mobilenet_v2
  image_pyramid: None
  add_image_level_feature: True
  image_pooling_crop_size: None
  image_pooling_stride: ['1', '1']
  aspp_with_batch_norm: True
  aspp_with_separable_conv: True
  multi_grid: [1, 2, 4]
  depth_multiplier: 1.0
  divisible_by: None
  decoder_output_stride: None
  decoder_use_separable_conv: True
  merge_method: max
  prediction_with_upsampled_logits: True
  dense_prediction_cell_json: 
  nas_stem_output_num_conv_filters: 20
  nas_use_classification_head: False
  nas_remove_os32_stride: False
  use_bounded_activation: False
  aspp_with_concat_projection: True
  aspp_with_squeeze_and_excitation: False
  aspp_convs_filters: 256
  decoder_use_sum_merge: False
  decoder_filters: 256
  decoder_output_is_logits: False
  image_se_uses_qsigmoid: False
  label_weights: None
  batch_norm_decay: 0.9997
  f: /home/lis-paul/.local

## load sample dataset

In [5]:
PATH = '/home/lis-paul/data/dsb2018/dsb2018_sub1/'
X_trn, Y_trn, X_val, Y_val = dataloader.load_img_dir(PATH)

number of images: 292
- training:       248
- validation:      44


In [6]:
# ranf=list(np.random.ranf(8)+10*np.random.ranf(1))
AUG = Compose([
#         CoarseDropout(max_holes=64, max_height=8, max_width=8, min_holes=None, min_height=None, min_width=None, 
#                   fill_value=ranf, always_apply=False, p=.9),
        GaussNoise(var_limit=(0.0, 0.05), mean=0, p=.5),
        GaussianBlur(blur_limit=3, p=.5),
        Flip(p=0.5),
        ShiftScaleRotate(shift_limit=0, scale_limit=(0.2, 1), rotate_limit=15, 
                         interpolation=0, border_mode=cv2.BORDER_REFLECT_101, value=0, mask_value=0, p=1),
        ElasticTransform(alpha=100, sigma=10, alpha_affine=1, p=0.7, 
                         interpolation=0, border_mode=cv2.BORDER_REFLECT_101, value=0, mask_value=0),
        RandomCrop(256, 256, always_apply=True, p=1.0)
    ], p=0.9)

AUG_val = RandomCrop(256, 256, always_apply=True, p=1.0)

In [7]:
train_dl = dataloader.Dataloader(X_trn, Y_trn, batch_size=8, patch_size=(256,256), augmenter=AUG, shuffle=True)
# val_dl   = dataloader.Dataloader(X_val, Y_val, batch_size=16, patch_size=(256,256), augmenter=AUG_val, shuffle=False)

xx,yy = train_dl[0]
# print(xx.dtype, yy[0].dtype, yy[1].dtype, yy[2].dtype)
# print(xx.shape, yy[0].shape, yy[1].shape, yy[2].shape)
# print(np.max(xx), np.max(yy[0]), yy[1].dtype, yy[2].dtype)


In [8]:
np.min(yy[2]), np.max(yy[2])

(-0.9993363229371175, 0.9993919671255781)

In [9]:
# train_generator = SegmentationMultiGenerator(datasets, folder) # My keras.utils.sequence object

def generator():
    multi_enqueuer = OrderedEnqueuer(train_dl, use_multiprocessing=True)
    multi_enqueuer.start(workers=8, max_queue_size=16)
    while True:
        xx, yy = next(multi_enqueuer.get())
        yield xx[...,np.newaxis], yy[0,...,np.newaxis], np.stack((yy[1], yy[2]),axis=-1)

## build network and get losses

In [10]:
dataset = tf.data.Dataset.from_generator(generator,
                                 output_types=(tf.uint8, tf.float32, tf.float32),
                                 output_shapes=(tf.TensorShape([None, 256, 256, 1]),
                                                tf.TensorShape([None, 256, 256, 1]),
                                                tf.TensorShape([None, 256, 256, 2]))
                                )

itr = dataset.make_one_shot_iterator()
sample = itr.get_next()

logits, prob_loss, grad_loss = modelbuilder.my_model(sample,
                                    model_options,
                                    weight_decay=FLAGS.weight_decay,
                                    is_training=True,
                                    fine_tune_batch_norm=FLAGS.fine_tune_batchnorm)


## learning rate finder

In [11]:
# FLAGS.training_number_of_steps = 500
# gs = tf.train.get_or_create_global_step()
# momentum = FLAGS.momentum

# learning_rate = lr_schedular.learning_rate_range_test(gs, FLAGS.training_number_of_steps, 1e-5, 1e-5)

## setup learning rate and optimizer

In [12]:
gs = tf.train.get_or_create_global_step()
ss = (FLAGS.training_number_of_steps * 0.9)/2

# update_op = tf.assign_add(gs, 1)
# with tf.control_dependencies([update_op]):
learning_rate = lr_schedular.cyclic_learning_rate(gs,
                                                  learning_rate=2e-5,
                                                  max_lr=1e-3,
                                                  step_size=ss,
                                                  max_steps=FLAGS.training_number_of_steps,
                                                  scale_rate=0.9,
                                                  mode='triangular',
                                                  policy='one_cycle',
                                                  name='oclr')

momentum = lr_schedular.cyclical_momentum(  gs,
                                            min_momentum=0.95,
                                            max_momentum=0.98,
                                            step_size=ss)

In [13]:
# learning_rate = train_utils.get_model_learning_rate(
#   FLAGS.learning_policy,
#   FLAGS.base_learning_rate,
#   FLAGS.learning_rate_decay_step,
#   FLAGS.learning_rate_decay_factor,
#   FLAGS.training_number_of_steps,
#   FLAGS.learning_power,
#   FLAGS.slow_start_step,
#   FLAGS.slow_start_learning_rate,
#   decay_steps=FLAGS.decay_steps,
#   end_learning_rate=FLAGS.end_learning_rate)
# gs = tf.train.get_or_create_global_step()
# momentum = FLAGS.momentum


In [14]:
# g = tf.get_default_graph()
# with g.as_default():
#     with tf.Session(graph=g) as sess:
#         sess.run(tf.global_variables_initializer())
#         for n in range(0,FLAGS.training_number_of_steps):
#             gs, lr = sess.run([global_step, learning_rate])
            
#             print(lr, gs)
#             writer = tf.summary.FileWriter("output3/lr2", sess.graph)
#             tf.summary.scalar('lr', lr)
#             update_op = tf.assign_add(global_step, 1)
        
#         writer.close()

## hard pixels mining

In [15]:
hard_example_mining_step = tf.cast(tf.math.ceil(FLAGS.training_number_of_steps * 0.8), tf.int64)
top_k_percent_pixels = 0.5
grad_loss_sum = tf.losses.compute_weighted_loss(grad_loss)
grad_loss_flatten = tf.reshape(grad_loss, shape=[-1])

s1 = tf.to_float(tf.shape(grad_loss_flatten)[0])
ratio = tf.minimum(1.0, tf.cast(gs / hard_example_mining_step, tf.float32))
top_k_pixels = tf.to_int32((ratio * top_k_percent_pixels + (1.0 - ratio)) * s1 )
top_k_losses, _ = tf.nn.top_k(grad_loss_flatten,
                              k=top_k_pixels,
                              sorted=True,
                              name='top_k_percent_pixels')
grad_loss_topk = tf.reduce_sum(top_k_losses) / tf.cast(top_k_pixels, tf.float32)

In [16]:
# s2 = tf.shape(grad_loss)
# s3 = tf.shape(grad_loss_flatten)
# s4 = tf.shape(top_k_losses)

    
# g = tf.get_default_graph()
# with g.as_default():
#     with tf.Session(graph=g) as sess:
#         sess.run(tf.global_variables_initializer())
#         for e in range(1,100):
# #             sess.run([train_op, check_op])
#             gs2, lr = sess.run([gs, learning_rate])
#             print(lr, gs2)
#             writer = tf.summary.FileWriter("output3/lr2", sess.graph)
#             tf.summary.scalar('top_k_pixels', top_k_pixels)
# #             tf.assign_add(gs, 1)
#             print(s1.eval(), s2.eval(), s3.eval(), grad_loss_sum.eval(), ratio.eval(), top_k_pixels.eval(), s4.eval(), grad_loss_topk.eval())

In [17]:
# ratio = tf.minimum(1.0, tf.cast(gs / hard_example_mining_step, tf.float32))
# top_k_pixels = tf.to_int32(ratio * top_k_percent_pixels + (1.0 - ratio))  ## don't multiply s1
# top_k_losses, _ = tf.nn.top_k(grad_loss_flatten,
#                               k=top_k_pixels,
#                               sorted=True,
#                               name='top_k_percent_pixels')
# grad_loss_topk = tf.reduce_sum(top_k_losses)



total_loss = tf.add(prob_loss , grad_loss_topk * 20)
tf.losses.add_loss(total_loss)
tf.losses.get_losses()

[<tf.Tensor 'prob/weighted_loss/value:0' shape=() dtype=float32>,
 <tf.Tensor 'grad/mse_grad/Mul:0' shape=(?, 256, 256, 2) dtype=float32>,
 <tf.Tensor 'weighted_loss/value:0' shape=() dtype=float32>,
 <tf.Tensor 'Add_1:0' shape=() dtype=float32>]

In [18]:
if FLAGS.optimizer == 'momentum':
    optimizer = tf.train.MomentumOptimizer(learning_rate, momentum)
elif FLAGS.optimizer == 'adam':
    optimizer = tf.train.AdamOptimizer(
    learning_rate=FLAGS.adam_learning_rate, epsilon=FLAGS.adam_epsilon)
else:
    raise ValueError('Unknown optimizer')

optimizer

In [19]:
# # Retrieve update ops for batch normalization
# update_ops = tf.GraphKeys.UPDATE_OPS
# # Define optimizer for training
# with tf.control_dependencies(tf.get_collection(update_ops)):
#     optimizer.minimize(total_loss)

## add summaries

In [20]:
summaries = set(tf.get_collection(tf.GraphKeys.SUMMARIES))

In [21]:
summaries

{<tf.Tensor 'grad_loss_pix:0' shape=() dtype=string>,
 <tf.Tensor 'prob_loss_pix:0' shape=() dtype=string>,
 <tf.Tensor 'prob_pred:0' shape=() dtype=string>}

In [22]:
summaries.add(tf.summary.scalar('prob_loss', prob_loss))
summaries.add(tf.summary.scalar('grad_loss', grad_loss_sum))
summaries.add(tf.summary.scalar('total_loss', total_loss))
summaries.add(tf.summary.scalar('learning_rate', learning_rate))
summaries.add(tf.summary.scalar('momentum', momentum))
summaries.add(tf.summary.scalar('topk_percent', top_k_pixels))
summaries.add(tf.summary.scalar('grad_loss_topk', grad_loss_topk))

tensor_name = logits.op.name
# summaries.add(tf.summary.histogram(tensor_name + '/activation', logits))

In [23]:
list(summaries)

[<tf.Tensor 'prob_pred:0' shape=() dtype=string>,
 <tf.Tensor 'prob_loss_pix:0' shape=() dtype=string>,
 <tf.Tensor 'grad_loss:0' shape=() dtype=string>,
 <tf.Tensor 'prob_loss:0' shape=() dtype=string>,
 <tf.Tensor 'grad_loss_pix:0' shape=() dtype=string>,
 <tf.Tensor 'total_loss:0' shape=() dtype=string>,
 <tf.Tensor 'learning_rate:0' shape=() dtype=string>,
 <tf.Tensor 'momentum:0' shape=() dtype=string>,
 <tf.Tensor 'topk_percent:0' shape=() dtype=string>,
 <tf.Tensor 'grad_loss_topk:0' shape=() dtype=string>]

In [24]:
summary_op = tf.summary.merge(list(summaries))

## setup train_op

In [25]:
train_op = contrib_slim.learning.create_train_op(
            total_loss,
            optimizer,
            gs,
            summarize_gradients=False,
            clip_gradient_norm=4)

## define tf session config

In [26]:
session_config = tf.ConfigProto(
        allow_soft_placement=True, 
        log_device_placement=False)
        
# session_config.gpu_options.allow_growth = True,
# session_config.gpu_options.per_process_gpu_memory_fraction = 0.8 

from datetime import datetime

now = datetime.now()
nowstr = now.strftime("%Y%m%d-%H:%M:%S")

In [27]:
# check_op = tf.add_check_numerics_ops()

# g = tf.get_default_graph()
# with g.as_default():
#     with tf.Session(graph=g) as sess:
#         sess.run(tf.global_variables_initializer())
#         for e in range(1,500):
#             sess.run([train_op, check_op])
#             print(gs.eval(), learning_rate.eval())

## start training

In [28]:
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=RuntimeWarning)
    contrib_slim.learning.train(
          train_op,
          logdir=f"slim_output/{nowstr}_oclr+m95-98_wd1e-4_g*20_topk50",
          log_every_n_steps=50,
          number_of_steps=FLAGS.training_number_of_steps,
          global_step=gs,
          session_config=session_config,
          summary_op=summary_op,
          save_summaries_secs=5)

Instructions for updating:
Please switch to tf.train.MonitoredTrainingSession
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Starting Session.
INFO:tensorflow:Saving checkpoint to path slim_output/20200619-23:22:29_oclr+m95-98_wd1e-4_g*20_topk50/model.ckpt
INFO:tensorflow:Starting Queues.
INFO:tensorflow:global_step/sec: 0
INFO:tensorflow:Recording summary at step 0.
INFO:tensorflow:global_step/sec: 0
INFO:tensorflow:Recording summary at step 0.
INFO:tensorflow:global_step/sec: 4.02949
INFO:tensorflow:Recording summary at step 20.
INFO:tensorflow:global_step/sec: 4.18459
INFO:tensorflow:Recording summary at step 40.
INFO:tensorflow:global step 49: loss = 3.2111 (0.213 sec/step)
INFO:tensorflow:global_step/sec: 4.01291
INFO:tensorflow:Recording summary at step 60.
INFO:tensorflow:Recording summary at step 84.
INFO:tensorflow:global_step/sec: 4.54473
INFO:tensorflow:global step 99: loss = 1.6898 (0.213 sec/step)
INFO:tensorflow:Recordin

In [29]:
# g = tf.get_default_graph()
# with g.as_default():
#     with tf.Session(graph=g) as sess:
# #         img = tf.random_uniform(
# #                 (4, crop_size[0], crop_size[1], 1))
# #         img = tf.convert_to_tensor(xx[...,np.newaxis], dtype=tf.uint8)
# #         print(img.shape)
# #         print(img.dtype)
# #         outputs_to_scales_to_logits = model.multi_scale_logits(
# #                                         inputs,
# #                                         model_options,
# #                                         image_pyramid=[1.0])

#         sess.run(tf.global_variables_initializer())
# #         outputs_to_scales_to_logits = sess.run(outputs_to_logits, feed_dict={'input_layer:0': img.eval(), 'training:0':True})
# #         outputs = sess.run(outputs_scaled, feed_dict={'input_layer:0': img.eval(), 'training:0':True})
# #         output0 = sess.run(logits)
#         ploss = sess.run(prob_loss)
#         gloss = sess.run(grad_loss)
#         tloss = sess.run(total_loss)
#         print(ploss.shape, gloss.shape, tloss.shape)
# #         print((output0['semantic']['merged_logits']).shape)
#         writer = tf.summary.FileWriter("output3/total", sess.graph)
    
# #         tf.summary.scalar('yaw_total_loss', yaw_total_loss)
# #         tf.summary.scalar('pitch_total_loss', pitch_total_loss)
# #         tf.summary.scalar('roll_total_loss', roll_total_loss)
    
#         writer.close()